## STEP 1: Brand Mapping 

In [2]:
import os
import re
import math
import warnings
import unicodedata
from typing import List, Tuple, Dict
from collections import defaultdict
import pandas as pd
import numpy as np

# Quick fix for MKL/sklearn crashes
os.environ['MKL_THREADING_LAYER'] = 'GNU'

warnings.filterwarnings("ignore")

# ---------------- CONFIG ----------------
CONFIG = {
    "input_file": "NUTRACEUTICALS AND NUTRITION combined_data_ June - August 2025.xlsx",
    "brands_file": "Brands.xlsx",
    "output_file": "NUTRACEUTICALS_Enhanced_with_Brands_and_Clusters.xlsx",
    "use_dedup": True,
    "distance_threshold": 0.50,
    "embedding_model": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "fallback_fuzzy_threshold": 70,
    "max_top_keywords_per_cluster": 12,
    "embedding_batch_size": 64,
    "brand_cluster_min_size": 3,
    "brand_similarity_threshold": 0.6,
    "high_fuzzy_threshold": 90,  # Increased for stricter matching
    "med_fuzzy_threshold": 85,   # Increased for stricter matching
    "low_fuzzy_threshold": 80,   # Increased for stricter matching
    "min_brand_length": 3,
    "simple_clustering_threshold": 1000,
    "chunk_size": 1000,
    "exact_match_threshold": 95,  # New: for very strict exact matching
    "partial_match_penalty": 0.3,  # New: penalty for partial matches
    "min_confidence_threshold": 0.75  # New: minimum confidence to accept match
}

# ---------------- NORMALIZATION FUNCTIONS ----------------

ARABIC_VARIATIONS = {
    'أ': 'ا', 'إ': 'ا', 'آ': 'ا', 'ٱ': 'ا',
    'ي': 'ي', 'ى': 'ي', 'ئ': 'ي', 'ؤ': 'و',
    'ة': 'ه',
    'پ': 'ب', 'چ': 'ج', 'ژ': 'ز', 'گ': 'ك', 'ک': 'ك',
    'ك': 'ك', 'ک': 'ك',
    'ی': 'ي', 'ي': 'ي'
}

ALNUM_PATTERN = re.compile(r"[^0-9a-zA-Z\u0600-\u06FF\u0750-\u077F]+", re.UNICODE)

def enhanced_arabic_normalize(text: str) -> str:
    if not isinstance(text, str):
        return ""
    for old, new in ARABIC_VARIATIONS.items():
        text = text.replace(old, new)
    text = re.sub(r'[\u064B-\u0652\u0670\u0640]', '', text)
    return text

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = enhanced_arabic_normalize(text)
    text = text.strip().lower()
    text = ALNUM_PATTERN.sub(" ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tight_normalize(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = enhanced_arabic_normalize(text)
    text = text.lower()
    text = ALNUM_PATTERN.sub("", text)
    return text

def enhanced_clustering_normalize(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = normalize_text(s)
    s = re.sub(r'\b(for|with|and|or|&|mg|g|ml|oz|units?|u|gm)\b', ' ', s)
    s = re.sub(r'\b(size|pack|piece|pcs|box|tin|can|container|vial)\b', ' ', s)
    s = re.sub(r'\b(stage|step|phase|level|new)\b', ' ', s)
    s = re.sub(r'\d+(\.\d+)?', ' NUM ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# ---------------- IMPORT OPTIONAL LIBRARIES ----------------

try:
    from sentence_transformers import SentenceTransformer
    EMBEDDINGS_OK = True
except ImportError:
    EMBEDDINGS_OK = False

try:
    from rapidfuzz import fuzz, process
    USE_RAPIDFUZZ = True
except ImportError:
    import difflib
    USE_RAPIDFUZZ = False

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.cluster import AgglomerativeClustering
except ImportError:
    TfidfVectorizer = None
    AgglomerativeClustering = None

# ---------------- ENHANCED BRAND MAPPING ----------------

def create_enhanced_brand_mapping():
    """Create comprehensive brand mapping with Arabic names and product lines"""
    
    # Enhanced brand mapping with Arabic transliterations and product lines
    enhanced_mapping = {
        # ARGIVIT - Main brand with product lines
        "ARGIVIT": [
            "argivit", "ارجيفيت", "ارجیفت", "ارجفیت", "ارجیفت",
            "argivit classic", "argivit focus", "argivit immune",
            "argivit smart", "argivit growth"
        ],
        
        # VITABIOTICS - Include Arabic names
        "VITABIOTICS": [
            "vitabiotics", "فیتابیوتکس", "فيتابيوتكس",
            "osteocare", "feroglobin", "فیروجلوبین", "فيروجلوبين",
            "pregnacare", "wellwoman", "wellman"
        ],
        
        # CENTRUM - Include Arabic variations
        "CENTRUM": [
            "centrum", "سنتروم", "سینتروم", "سنترم"
        ],
        
        # NOW Foods
        "NOW Foods": [
            "now foods", "now", "ناو", "ناو فودز"
        ],
        
        # Manuka Health
        "Manuka Health": [
            "manuka health", "manuka", "مانوکا", "مانوكا"
        ],
        
        # Foods Alive
        "Foods Alive": [
            "foods alive", "alive"
        ],
        
        # Stevia (if it's a brand, not just ingredient)
        "Stevia": [
            "stevia", "ستیفیا", "ستيفيا", "سکر ستیفیا"
        ]
    }
    
    return enhanced_mapping

def match_brand_with_priority(query: str, brand_mappings: dict) -> tuple:
    """Match brand with priority rules to avoid misclassification"""
    
    query_norm = normalize_text(query)
    best_match = None
    best_confidence = 0
    
    # Priority 1: Exact brand name match (highest priority)
    for brand, variations in brand_mappings.items():
        for variation in variations:
            variation_norm = normalize_text(variation)
            
            # Exact match check
            if variation_norm == query_norm:
                return brand, 1.0, "exact_match"
            
            # Word boundary exact match
            if f" {variation_norm} " in f" {query_norm} " or \
               query_norm.startswith(f"{variation_norm} ") or \
               query_norm.endswith(f" {variation_norm}"):
                confidence = 0.95
                if confidence > best_confidence:
                    best_match = brand
                    best_confidence = confidence
    
    # Priority 2: Product line disambiguation
    # If "argivit focus" found, assign to ARGIVIT, not Focus
    if "argivit" in query_norm and "focus" in query_norm:
        return "ARGIVIT", 0.9, "product_line_match"
    
    # Priority 3: Prevent generic word matches
    generic_terms = ["omega", "اومیجا", "vitamin", "فیتامین"]
    for term in generic_terms:
        if normalize_text(term) == query_norm:
            return "Other", 0.0, "generic_term"
    
    # Priority 4: Fuzzy matching with strict thresholds
    if best_match and best_confidence >= 0.75:
        return best_match, best_confidence, "fuzzy_match"
    
    return "Other", 0.0, "no_match"

def load_brands_from_file(brands_file: str) -> Dict[str, str]:
    try:
        brands_df = pd.read_excel(brands_file, sheet_name='Brands')
        
        # Start with enhanced mapping
        enhanced_mapping = create_enhanced_brand_mapping()
        brand_mapping = {}
        
        # Add enhanced mappings first
        for brand, variations in enhanced_mapping.items():
            for variation in variations:
                brand_mapping[normalize_text(variation)] = brand
                brand_mapping[tight_normalize(variation)] = brand
        
        # Store original brand names for exact matching
        exact_brand_names = {}
        
        for _, row in brands_df.iterrows():
            brand_en = str(row['Brand EN']).strip()
            brand_ar = str(row['Brand AR']).strip()
            
            if brand_en and brand_en != 'nan':
                # Store exact matches (case-insensitive but preserve original case)
                exact_brand_names[brand_en.lower()] = brand_en
                exact_brand_names[tight_normalize(brand_en)] = brand_en
                
                # Store normalized versions
                brand_mapping[brand_en.lower()] = brand_en
                brand_mapping[tight_normalize(brand_en)] = brand_en
                brand_mapping[normalize_text(brand_en)] = brand_en
                
            if brand_ar and brand_ar != 'nan':
                # Store exact matches for Arabic
                exact_brand_names[brand_ar] = brand_en
                exact_brand_names[tight_normalize(brand_ar)] = brand_en
                exact_brand_names[normalize_text(brand_ar)] = brand_en
                
                # Store normalized versions
                brand_mapping[brand_ar] = brand_en
                brand_mapping[tight_normalize(brand_ar)] = brand_en
                brand_mapping[normalize_text(brand_ar)] = brand_en
        
        print(f"✓ Loaded {len(set(brands_df['Brand EN'].dropna()))} brands from file")
        print(f"✓ Enhanced with {len(enhanced_mapping)} predefined brand mappings")
        return brand_mapping, exact_brand_names
    except Exception as e:
        print(f"Warning: Could not load brands file ({e}), using enhanced mapping only")
        enhanced_mapping = create_enhanced_brand_mapping()
        brand_mapping = {}
        exact_brand_names = {}
        
        for brand, variations in enhanced_mapping.items():
            for variation in variations:
                brand_mapping[normalize_text(variation)] = brand
                brand_mapping[tight_normalize(variation)] = brand
                exact_brand_names[variation] = brand
                
        return brand_mapping, exact_brand_names

def load_descriptions_from_file(brands_file: str) -> List[str]:
    try:
        desc_df = pd.read_excel(brands_file, sheet_name='Description')
        descriptions = desc_df.iloc[:, 0].dropna().astype(str).tolist()
        print(f"✓ Loaded {len(descriptions)} product descriptions")
        return descriptions
    except Exception as e:
        print(f"Note: Could not load descriptions ({e})")
        return []

def build_brand_word_index(brand_mapping: dict) -> dict:
    """Build index of brand words to avoid partial matches"""
    brand_words = set()
    for brand_key, brand_name in brand_mapping.items():
        # Add full brand name words
        words = normalize_text(brand_name).split()
        for word in words:
            if len(word) >= CONFIG["min_brand_length"]:
                brand_words.add(word)
    return brand_words

def build_nutraceutical_brand_catalog(brands_file: str):
    file_brands, exact_brand_names = load_brands_from_file(brands_file)
    descriptions = load_descriptions_from_file(brands_file)
    
    # Get canonical brands from enhanced mapping
    enhanced_mapping = create_enhanced_brand_mapping()
    canonical_brands = list(enhanced_mapping.keys())
    
    # Add brands from file
    canonical_brands.extend(list(set(file_brands.values())))
    canonical_brands = list(set(canonical_brands))  # Remove duplicates
    canonical_brands.append("Other")
    
    # Build exact match patterns
    exact_matches = {}
    word_to_brand = {}  # Map individual words to their brands
    regex_patterns = []
    
    # Process enhanced mappings first
    for brand_name, variations in enhanced_mapping.items():
        for variation in variations:
            tight = tight_normalize(variation)
            if tight and len(tight) >= CONFIG["min_brand_length"]:
                exact_matches[tight] = brand_name
                
            # Create regex patterns for exact word boundary matching
            if len(variation) >= CONFIG["min_brand_length"]:
                escaped = re.escape(variation)
                pattern = re.compile(rf"\b{escaped}\b", re.IGNORECASE | re.UNICODE)
                regex_patterns.append((pattern, brand_name))
                
            # Map individual words to brands (for multi-word brands)
            words = normalize_text(variation).split()
            if len(words) == 1 and len(words[0]) >= CONFIG["min_brand_length"]:
                word_to_brand[words[0]] = brand_name
    
    # Process file brands
    for brand_key, brand_name in file_brands.items():
        tight = tight_normalize(brand_key)
        if tight and len(tight) >= CONFIG["min_brand_length"]:
            exact_matches[tight] = brand_name
            
        # Create regex patterns for exact word boundary matching
        if len(brand_key) >= CONFIG["min_brand_length"]:
            escaped = re.escape(brand_key)
            pattern = re.compile(rf"\b{escaped}\b", re.IGNORECASE | re.UNICODE)
            regex_patterns.append((pattern, brand_name))
            
        # Map individual words to brands (for multi-word brands)
        words = normalize_text(brand_key).split()
        if len(words) == 1 and len(words[0]) >= CONFIG["min_brand_length"]:
            word_to_brand[words[0]] = brand_name
    
    # Build brand word index to avoid false positives
    brand_words = build_brand_word_index(file_brands)
    
    catalog = {
        "canonical": canonical_brands,
        "exact_matches": exact_matches,
        "exact_brand_names": exact_brand_names,
        "word_to_brand": word_to_brand,
        "regex_patterns": regex_patterns,
        "file_brands": file_brands,
        "brand_words": brand_words,
        "enhanced_mapping": enhanced_mapping
    }
    return catalog

# ---------------- ENHANCED BRAND MATCHING FUNCTIONS ----------------

def is_exact_brand_match(query: str, brand_name: str) -> bool:
    """Check if query contains exact brand name"""
    query_norm = normalize_text(query)
    brand_norm = normalize_text(brand_name)
    
    # Check if brand name appears as complete word(s) in query
    brand_words = brand_norm.split()
    query_words = query_norm.split()
    
    if len(brand_words) == 1:
        # Single word brand - must appear as complete word
        return brand_words[0] in query_words
    else:
        # Multi-word brand - check if all words appear consecutively
        brand_text = " ".join(brand_words)
        return brand_text in query_norm

def calculate_enhanced_confidence(query: str, matched_brand: str, match_type: str, score: float = 0) -> float:
    """Enhanced confidence calculation with stricter rules"""
    base_confidence = 0.0
    
    if match_type == "exact_full":
        base_confidence = 1.0
    elif match_type == "exact_word":
        base_confidence = 0.95
    elif match_type == "regex_exact":
        base_confidence = 0.9
    elif match_type == "product_line_match":
        base_confidence = 0.9
    elif match_type == "corrected_match":
        base_confidence = 0.95
    elif match_type == "arabic_match":
        base_confidence = 0.95
    elif match_type == "fuzzy_very_high":
        base_confidence = min(0.85, score / 100)
    elif match_type == "fuzzy_high":
        base_confidence = min(0.75, score / 100)
    else:
        base_confidence = 0.5
    
    # Apply penalties for partial matches
    query_words = set(normalize_text(query).split())
    brand_words = set(normalize_text(matched_brand).split())
    
    # If query contains many non-brand words, reduce confidence
    if len(query_words) > len(brand_words) * 2:
        base_confidence *= 0.8
    
    # If brand name is much shorter than query, it might be coincidental
    brand_length = len(normalize_text(matched_brand).replace(" ", ""))
    query_length = len(normalize_text(query).replace(" ", ""))
    
    if brand_length < query_length * 0.3:  # Brand is less than 30% of query length
        base_confidence *= CONFIG["partial_match_penalty"]
    
    return base_confidence

def enhanced_brand_match(query: str, catalog: dict) -> tuple:
    """Enhanced brand matching with stricter rules and special fixes"""
    if not isinstance(query, str) or not query.strip():
        return "Other", 0.0
    
    original_query = query.strip()
    norm_query = normalize_text(original_query)
    tight_query = tight_normalize(original_query)
    query_words = norm_query.split()
    query_lower = original_query.lower()
    
    matches = []
    
    # SPECIAL CASE FIXES FIRST (highest priority)
    
    # Fix: ARGIVIT Focus should be ARGIVIT
    if any(term in query_lower for term in ['argivit focus', 'ارجفیت', 'ارجیفت']):
        confidence = calculate_enhanced_confidence(original_query, "ARGIVIT", "corrected_match")
        return "ARGIVIT", confidence
    
    # Fix: Generic Omega-3 should be Other
    if any(term in query_lower for term in ['اومیجا 3', 'اومیجا3']) and 'jamjoom' not in query_lower:
        return "Other", 0.0
    
    # Fix: Arabic VITABIOTICS
    if 'فیتابیوتکس' in original_query:
        confidence = calculate_enhanced_confidence(original_query, "VITABIOTICS", "arabic_match")
        return "VITABIOTICS", confidence
    
    # Check enhanced mappings first
    enhanced_mapping = catalog.get("enhanced_mapping", {})
    for brand, variations in enhanced_mapping.items():
        for variation in variations:
            variation_norm = normalize_text(variation)
            
            # Exact match check
            if variation_norm == norm_query:
                confidence = calculate_enhanced_confidence(original_query, brand, "exact_full")
                matches.append((brand, confidence, "exact_full"))
                
            # Word boundary exact match
            elif f" {variation_norm} " in f" {norm_query} " or \
                 norm_query.startswith(f"{variation_norm} ") or \
                 norm_query.endswith(f" {variation_norm}"):
                confidence = calculate_enhanced_confidence(original_query, brand, "exact_word")
                matches.append((brand, confidence, "exact_word"))
    
    # 1. EXACT TIGHT NORMALIZATION MATCH
    if tight_query in catalog["exact_matches"]:
        brand = catalog["exact_matches"][tight_query]
        confidence = calculate_enhanced_confidence(original_query, brand, "exact_full")
        matches.append((brand, confidence, "exact_full"))
    
    # 2. EXACT BRAND NAME MATCH (check if brand appears exactly in query)
    for brand_key, brand_name in catalog["exact_brand_names"].items():
        if is_exact_brand_match(original_query, brand_name):
            confidence = calculate_enhanced_confidence(original_query, brand_name, "exact_word")
            matches.append((brand_name, confidence, "exact_word"))
    
    # 3. REGEX PATTERN MATCHING (word boundaries)
    for pattern, brand in catalog["regex_patterns"]:
        if pattern.search(original_query):
            confidence = calculate_enhanced_confidence(original_query, brand, "regex_exact")
            matches.append((brand, confidence, "regex_exact"))
    
    # 4. SINGLE WORD BRAND MATCHING (only for single-word queries or exact word matches)
    if len(query_words) == 1:  # Only for single word queries
        single_word = query_words[0]
        if single_word in catalog["word_to_brand"]:
            brand = catalog["word_to_brand"][single_word]
            confidence = calculate_enhanced_confidence(original_query, brand, "exact_word")
            matches.append((brand, confidence, "exact_word"))
    else:
        # For multi-word queries, check if any single word is an exact brand match
        for word in query_words:
            if word in catalog["word_to_brand"]:
                brand = catalog["word_to_brand"][word]
                # Only accept if the word is significant part of the query
                if len(word) >= len(norm_query) * 0.4:  # Word is at least 40% of query length
                    confidence = calculate_enhanced_confidence(word, brand, "exact_word") * 0.8
                    matches.append((brand, confidence, "exact_word"))
    
    # 5. FUZZY MATCHING (very strict, only for high scores)
    if USE_RAPIDFUZZ and not matches:
        try:
            # Only match against exact brand names
            brand_candidates = list(catalog["exact_brand_names"].values())
            brand_candidates.extend([key for key in catalog["exact_brand_names"].keys() if isinstance(key, str)])
            
            if brand_candidates:
                result = process.extractOne(
                    norm_query,
                    brand_candidates,
                    scorer=fuzz.token_sort_ratio,  # More strict than token_set_ratio
                    score_cutoff=CONFIG["exact_match_threshold"]  # Very high threshold
                )
                
                if result:
                    matched_text, score, _ = result
                    # Find the brand name for this match
                    brand = catalog["exact_brand_names"].get(matched_text, matched_text)
                    if brand in catalog["canonical"]:
                        match_type = "fuzzy_very_high" if score >= 95 else "fuzzy_high"
                        confidence = calculate_enhanced_confidence(original_query, brand, match_type, score)
                        matches.append((brand, confidence, match_type))
        except Exception:
            pass
    
    # 6. SELECT BEST MATCH
    if matches:
        # Sort by confidence and match type priority
        type_priority = {
            "exact_full": 0, "corrected_match": 1, "arabic_match": 2, "exact_word": 3, 
            "regex_exact": 4, "product_line_match": 5, "fuzzy_very_high": 6, "fuzzy_high": 7
        }
        matches.sort(key=lambda x: (-x[1], type_priority.get(x[2], 10)))
        
        best_brand, best_confidence, best_type = matches[0]
        
        # Apply minimum confidence threshold
        if best_confidence >= CONFIG["min_confidence_threshold"]:
            return best_brand, best_confidence
    
    return "Other", 0.0

# ---------------- CLUSTERING FUNCTIONS (UNCHANGED) ----------------

def cluster_brand_queries(brand_df: pd.DataFrame, start_cluster_id: int) -> pd.DataFrame:
    if len(brand_df) <= 1:
        brand_df = brand_df.copy()
        brand_df["Cluster ID"] = start_cluster_id
        return brand_df
    queries = brand_df["search_clean"].tolist()
    try:
        if EMBEDDINGS_OK:
            model = SentenceTransformer(CONFIG["embedding_model"])
            embeddings = model.encode(queries, batch_size=CONFIG["embedding_batch_size"], 
                                    show_progress_bar=False, normalize_embeddings=True)
            brand_threshold = min(CONFIG["distance_threshold"] * 0.8, 0.4)
            clustering = AgglomerativeClustering(
                n_clusters=None,
                distance_threshold=brand_threshold,
                metric='cosine',
                linkage='average'
            )
            labels = clustering.fit_predict(embeddings)
        elif USE_RAPIDFUZZ:
            n = len(queries)
            dist_matrix = np.zeros((n, n))
            for i in range(n):
                for j in range(i+1, n):
                    similarity = fuzz.token_set_ratio(queries[i], queries[j])
                    distance = 1 - (similarity / 100.0)
                    dist_matrix[i, j] = distance
                    dist_matrix[j, i] = distance
            clustering = AgglomerativeClustering(
                n_clusters=None,
                distance_threshold=0.3,
                metric='precomputed',
                linkage='average'
            )
            labels = clustering.fit_predict(dist_matrix)
        else:
            labels = []
            current_label = 0
            processed = set()
            for i, query in enumerate(queries):
                if i in processed:
                    continue
                cluster_members = [i]
                processed.add(i)
                for j, other_query in enumerate(queries[i+1:], i+1):
                    if j in processed:
                        continue
                    words1 = set(query.split())
                    words2 = set(other_query.split())
                    if len(words1 & words2) >= min(len(words1), len(words2)) * 0.6:
                        cluster_members.append(j)
                        processed.add(j)
                for idx in cluster_members:
                    if idx < len(labels):
                        labels[idx] = current_label
                    else:
                        labels.append(current_label)
                current_label += 1
            while len(labels) < len(queries):
                labels.append(current_label)
                current_label += 1
        labels = [label + start_cluster_id for label in labels]
    except Exception as e:
        print(f"    -> Warning: Brand clustering failed ({e}), using individual clusters")
        labels = list(range(start_cluster_id, start_cluster_id + len(brand_df)))
    result_df = brand_df.copy()
    result_df["Cluster ID"] = labels
    return result_df

def cluster_with_embeddings_offset(df: pd.DataFrame, offset: int) -> pd.DataFrame:
    queries = df["search_clean"].tolist()
    model = SentenceTransformer(CONFIG["embedding_model"])
    embeddings = model.encode(queries, batch_size=CONFIG["embedding_batch_size"], 
                            show_progress_bar=False, normalize_embeddings=True)
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=CONFIG["distance_threshold"],
        metric='cosine',
        linkage='average'
    )
    labels = clustering.fit_predict(embeddings)
    result_df = df.copy()
    result_df["Cluster ID"] = labels + offset
    return result_df

def cluster_with_tfidf_offset(df: pd.DataFrame, offset: int) -> pd.DataFrame:
    queries = df["search_clean"].tolist()
    vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=15000)
    X = vectorizer.fit_transform(queries)
    n_clusters = max(2, min(int(math.sqrt(len(queries))), len(queries)//2))
    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    labels = clustering.fit_predict(X.toarray())
    result_df = df.copy()
    result_df["Cluster ID"] = labels + offset
    return result_df

def cluster_with_fuzzy_offset(df: pd.DataFrame, offset: int) -> pd.DataFrame:
    queries = df["search_clean"].tolist()
    n = len(queries)
    if n <= 1:
        result_df = df.copy()
        result_df["Cluster ID"] = offset
        return result_df
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            if USE_RAPIDFUZZ:
                similarity = fuzz.partial_ratio(queries[i], queries[j])
            else:
                similarity = difflib.SequenceMatcher(None, queries[i], queries[j]).ratio() * 100
            distance = 1 - (similarity / 100.0)
            dist_matrix[i, j] = distance
            dist_matrix[j, i] = distance
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=1 - (CONFIG["fallback_fuzzy_threshold"]/100),
        metric='precomputed',
        linkage='average'
    )
    labels = clustering.fit_predict(dist_matrix)
    result_df = df.copy()
    result_df["Cluster ID"] = labels + offset
    return result_df

def perform_brand_aware_clustering(df: pd.DataFrame) -> Tuple[pd.DataFrame, str]:
    if df.empty:
        df["Cluster ID"] = []
        return df, "empty"
    if len(df) < CONFIG["simple_clustering_threshold"]:
        print(f"    -> Using simple clustering for small dataset ({len(df)} rows)")
        result_df = df.copy()
        result_df["Cluster ID"] = range(len(df))
        return result_df, "simple_identity"
    
    branded_df = df[df["Brand"] != "Other"].copy()
    other_df = df[df["Brand"] == "Other"].copy()
    results = []
    cluster_id_counter = 0
    
    if not branded_df.empty:
        print(f"    -> Clustering {len(branded_df)} branded queries...")
        for brand, brand_group in branded_df.groupby("Brand"):
            if len(brand_group) >= CONFIG["brand_cluster_min_size"]:
                brand_clustered = cluster_brand_queries(brand_group, cluster_id_counter)
                cluster_id_counter = brand_clustered["Cluster ID"].max() + 1
                results.append(brand_clustered)
            else:
                brand_group = brand_group.copy()
                brand_group["Cluster ID"] = range(cluster_id_counter, cluster_id_counter + len(brand_group))
                cluster_id_counter += len(brand_group)
                results.append(brand_group)
    
    if not other_df.empty:
        print(f"    -> Clustering {len(other_df)} unclassified queries...")
        try:
            if EMBEDDINGS_OK:
                other_clustered = cluster_with_embeddings_offset(other_df, cluster_id_counter)
                method = "brand_aware_embeddings"
            elif TfidfVectorizer is not None:
                other_clustered = cluster_with_tfidf_offset(other_df, cluster_id_counter)
                method = "brand_aware_tfidf"
            elif USE_RAPIDFUZZ:
                other_clustered = cluster_with_fuzzy_offset(other_df, cluster_id_counter)
                method = "brand_aware_fuzzy"
            else:
                other_clustered = other_df.copy()
                other_clustered["Cluster ID"] = range(cluster_id_counter, cluster_id_counter + len(other_df))
                method = "brand_aware_identity"
            results.append(other_clustered)
        except Exception as e:
            print(f"    -> Warning: Other clustering failed ({e}), using identity fallback")
            other_df_copy = other_df.copy()
            other_df_copy["Cluster ID"] = range(cluster_id_counter, cluster_id_counter + len(other_df))
            results.append(other_df_copy)
            method = "brand_aware_identity_fallback"
    else:
        method = "brand_aware_no_other"
    
    if results:
        final_df = pd.concat(results, ignore_index=True)
    else:
        final_df = df.copy()
        final_df["Cluster ID"] = range(len(final_df))
        method = "brand_aware_empty_fallback"
    
    return final_df, method

def process_nutraceutical_data_chunk(df: pd.DataFrame, brand_catalog: dict):
    df["search_original"] = df["search"].astype(str)
    df["search_clean"] = df["search_original"].apply(enhanced_clustering_normalize)
    
    print(f"  -> Mapping brands for {len(df)} queries...")
    brands = []
    brand_confidences = []
    
    for i, query in enumerate(df["search_original"]):
        if i % 1000 == 0 and i > 0:
            print(f"    -> Processed {i}/{len(df)} brand mappings...")
        
        brand, confidence = enhanced_brand_match(query, brand_catalog)
        brands.append(brand)
        brand_confidences.append(confidence)
    
    df["Brand"] = brands
    df["brand_confidence"] = brand_confidences
    
    if CONFIG["use_dedup"]:
        agg_cols = ["search_clean", "Brand"]
        base = df.groupby(agg_cols, as_index=False).agg({
            "count": "sum",
            "Clicks": "sum",
            "Conversions": "sum",
            "brand_confidence": "mean",
            "search_original": "first",
            "category": "first",
            "clickThroughRate": "mean",
            "conversionRate": "mean",
            "averageClickPosition": "mean",
            "underperforming": "first",
            "start_date": "first",
            "end_date": "first"
        })
    else:
        base = df.copy()
    
    clustered, method_used = perform_brand_aware_clustering(base)
    cluster_map = clustered[["search_clean", "Brand", "Cluster ID"]]
    df_merged = df.merge(cluster_map, on=["search_clean", "Brand"], how="left")
    
    queries_clustered = df_merged.sort_values(
        by=["Cluster ID", "Brand", "count"],
        ascending=[True, True, False]
    ).reset_index(drop=True)
    
    return {
        "method": method_used,
        "queries_clustered": queries_clustered,
        "clustered_base": clustered
    }

def process_nutraceutical_data(df: pd.DataFrame, brand_catalog: dict):
    results = []
    for i in range(0, len(df), CONFIG["chunk_size"]):
        chunk = df.iloc[i:i+CONFIG["chunk_size"]].copy()
        print(f"  -> Processing chunk {i//CONFIG['chunk_size'] + 1} ({len(chunk)} rows)...")
        chunk_result = process_nutraceutical_data_chunk(chunk, brand_catalog)
        results.append(chunk_result["queries_clustered"])
    
    final_df = pd.concat(results, ignore_index=True)
    
    return {
        "method": chunk_result["method"],
        "queries_clustered": final_df,
        "clustered_base": chunk_result["clustered_base"]
    }

def main():
    print("=== VALIDATING INPUT FILES ===")
    if not os.path.exists(CONFIG["input_file"]):
        raise FileNotFoundError(f"Input file not found: {CONFIG['input_file']}")
    if not os.path.exists(CONFIG["brands_file"]):
        raise FileNotFoundError(f"Brands file not found: {CONFIG['brands_file']}")
    print("✓ Input files validated successfully")
    
    print("\n=== BUILDING NUTRACEUTICALS BRAND CATALOG ===")
    try:
        brand_catalog = build_nutraceutical_brand_catalog(CONFIG["brands_file"])
        print(f"✓ Loaded {len(brand_catalog['canonical'])} canonical brands")
        print(f"✓ Built {len(brand_catalog['exact_matches'])} exact match patterns")
        print(f"✓ Built {len(brand_catalog['word_to_brand'])} word-to-brand mappings")
        print(f"✓ Built {len(brand_catalog['regex_patterns'])} regex patterns")
        print(f"✓ Enhanced with {len(brand_catalog['enhanced_mapping'])} predefined mappings")
    except Exception as e:
        raise ValueError(f"Error building brand catalog: {e}")
    
    print(f"\n=== LOADING NUTRACEUTICALS SEARCH DATA ===")
    try:
        df = pd.read_excel(CONFIG["input_file"])
        print(f"✓ Loaded {len(df)} search queries")
        
        print(f"\n=== PROCESSING DATA ===")
        result = process_nutraceutical_data(df, brand_catalog)
        print(f"✓ Method: {result['method']}")
        print(f"✓ Created {result['queries_clustered']['Cluster ID'].nunique()} clusters")
        
        brand_stats = result["queries_clustered"]["Brand"].value_counts()
        print(f"✓ Top brands: {dict(brand_stats.head(10))}")
        
    except Exception as e:
        raise ValueError(f"Error processing data: {e}")
    
    final_columns = [
        "category", "Brand", "search", "count", "Clicks", "Conversions", 
        "clickThroughRate", "conversionRate", "averageClickPosition", 
        "underperforming", "start_date", "end_date", "Cluster ID"
    ]
    
    final_df = result["queries_clustered"].copy()
    for col in final_columns:
        if col not in final_df.columns:
            if col == "Brand" or col == "Cluster ID":
                continue
            else:
                final_df[col] = None
    
    final_df = final_df[final_columns]
    
    print(f"\n=== SAVING RESULTS ===")
    try:
        with pd.ExcelWriter(CONFIG["output_file"], engine="xlsxwriter") as writer:
            final_df.to_excel(writer, sheet_name="enhanced_data", index=False)
            
            workbook = writer.book
            worksheet = writer.sheets["enhanced_data"]
            
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'top',
                'fg_color': '#D7E4BC',
                'border': 1
            })
            
            number_format = workbook.add_format({'num_format': '#,##0'})
            percent_format = workbook.add_format({'num_format': '0.00%'})
            
            for i, col in enumerate(final_df.columns):
                max_length = len(str(col))
                if len(final_df) > 0:
                    max_length = max(max_length, final_df[col].astype(str).str.len().max())
                max_length = min(max_length, 50)
                max_length = max(max_length, 10)
                worksheet.set_column(i, i, max_length)
                
                if col in ['count', 'Clicks', 'Conversions', 'Cluster ID']:
                    worksheet.set_column(i, i, None, number_format)
                elif col in ['clickThroughRate', 'conversionRate']:
                    worksheet.set_column(i, i, None, percent_format)
            
            worksheet.set_row(0, None, header_format)
        
        print(f"✓ Results saved to: {CONFIG['output_file']} (enhanced_data sheet only)")
        
    except Exception as e:
        print(f"❌ Error saving results with formatting: {e}")
        try:
            with pd.ExcelWriter(CONFIG["output_file"], engine="openpyxl") as writer:
                final_df.to_excel(writer, sheet_name="enhanced_data", index=False)
            print(f"✓ Results saved to: {CONFIG['output_file']} (basic format)")
        except Exception as e2:
            raise ValueError(f"Failed to save results: {e2}")
    
    print(f"\n=== FINAL STATISTICS ===")
    print(f"📊 Total queries processed: {len(final_df):,}")
    print(f"📊 Total clusters created: {final_df['Cluster ID'].nunique():,}")
    print(f"📊 Brands identified: {final_df['Brand'].nunique()}")
    
    brand_counts = final_df["Brand"].value_counts()
    total_queries = len(final_df)
    
    print(f"\n=== BRAND MAPPING RESULTS ===")
    print(f"🏷️  Branded queries: {len(final_df[final_df['Brand'] != 'Other']):,} ({len(final_df[final_df['Brand'] != 'Other'])/total_queries*100:.1f}%)")
    print(f"❓ Unclassified (Other): {len(final_df[final_df['Brand'] == 'Other']):,} ({len(final_df[final_df['Brand'] == 'Other'])/total_queries*100:.1f}%)")
    
    print(f"\n🏆 Top 15 brands by query count:")
    for i, (brand, count) in enumerate(brand_counts.head(15).items(), 1):
        percentage = count / total_queries * 100
        total_searches = final_df[final_df["Brand"] == brand]["count"].sum()
        print(f"  {i:2d}. {brand:<20}: {count:>4,} queries ({percentage:>5.1f}%) | {total_searches:>8,} searches")
    
    total_search_volume = final_df["count"].sum()
    print(f"\n=== SEARCH VOLUME STATISTICS ===")
    print(f"📈 Total search volume: {total_search_volume:,}")
    print(f"📈 Average searches per query: {total_search_volume/total_queries:.1f}")
    
    print(f"\n🔍 Top 10 queries by search volume:")
    top_queries = final_df.nlargest(10, "count")
    for i, (_, row) in enumerate(top_queries.iterrows(), 1):
        print(f"  {i:2d}. {row['search']:<30} [{row['Brand']:<15}]: {row['count']:>6,} searches")
    
    print(f"\n=== CLUSTER ANALYSIS ===")
    cluster_sizes = final_df.groupby('Cluster ID').size()
    print(f"📊 Average cluster size: {cluster_sizes.mean():.1f} queries")
    print(f"📊 Largest cluster: {cluster_sizes.max()} queries")
    print(f"📊 Smallest cluster: {cluster_sizes.min()} queries")
    
    brand_cluster_counts = final_df.groupby('Brand')['Cluster ID'].nunique().sort_values(ascending=False)
    print(f"\n📊 Top brands by cluster count:")
    for brand, cluster_count in brand_cluster_counts.head(10).items():
        query_count = len(final_df[final_df['Brand'] == brand])
        avg_cluster_size = query_count / cluster_count if cluster_count > 0 else 0
        print(f"  {brand:<20}: {cluster_count:>3} clusters | {query_count:>4} queries | avg size: {avg_cluster_size:.1f}")
    
    print(f"\n🎉 Processing completed successfully!")
    print(f"📁 Output file: {CONFIG['output_file']}")
    print(f"     - enhanced_data: Main data with Brand column and Cluster ID")
    print(f"\n📋 Column sequence in enhanced_data:")
    print(f"     {' | '.join(final_columns)}")

if __name__ == "__main__":
    main()

    


=== VALIDATING INPUT FILES ===
✓ Input files validated successfully

=== BUILDING NUTRACEUTICALS BRAND CATALOG ===
✓ Loaded 544 brands from file
✓ Enhanced with 7 predefined brand mappings
✓ Loaded 544 product descriptions
✓ Loaded 545 canonical brands
✓ Built 1060 exact match patterns
✓ Built 1060 word-to-brand mappings
✓ Built 1579 regex patterns
✓ Enhanced with 7 predefined mappings

=== LOADING NUTRACEUTICALS SEARCH DATA ===
✓ Loaded 15249 search queries

=== PROCESSING DATA ===
  -> Processing chunk 1 (1000 rows)...
  -> Mapping brands for 1000 queries...
    -> Using simple clustering for small dataset (338 rows)
  -> Processing chunk 2 (1000 rows)...
  -> Mapping brands for 1000 queries...
    -> Using simple clustering for small dataset (473 rows)
  -> Processing chunk 3 (1000 rows)...
  -> Mapping brands for 1000 queries...
    -> Using simple clustering for small dataset (565 rows)
  -> Processing chunk 4 (1000 rows)...
  -> Mapping brands for 1000 queries...
    -> Using sim

## STEP 2: Rearrange (Cluster)

In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Rearrange Nutraceuticals Search Queries by Word Similarity
-------------------------------------------------------
Purpose:
  - Load "NUTRACEUTICALS_Enhanced_with_Brands_and_Clusters.xlsx"
  - Process all sheets
  - Cluster similar search terms using multilingual embeddings (SentenceTransformer → Agglomerative)
  - Fallback to rapidfuzz partial_ratio for better grouping (e.g., "اومیجا ٣", "اومیغا ٣")
  - Sort by cluster_id (asc) and count (desc)
  - Add only cluster_id column, preserve all other columns unchanged
  - Handle Arabic/English variations (e.g., "میلاتونین", "melatonin")

Enhancements:
  - DISTANCE_THRESHOLD=0.5 for balanced clusters (fewer, meaningful groups)
  - Uses rapidfuzz.partial_ratio (threshold=70) for fuzzy fallback
  - No normalization or aggregation to preserve original terms and avoid errors
  - Robust error handling and logging

Requirements:
  pip install pandas numpy scikit-learn openpyxl sentence-transformers rapidfuzz

Usage:
  python rearrange_nutraceuticals.py
"""

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import logging
import os
import ssl
import urllib3

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Check if rapidfuzz is available
FUZZY_AVAILABLE = False
try:
    from rapidfuzz import fuzz
    FUZZY_AVAILABLE = True
    logger.info("rapidfuzz available for enhanced fallback matching.")
except ModuleNotFoundError:
    logger.warning("rapidfuzz not installed. Install with: pip install rapidfuzz")

# Define file paths
INPUT_FILE = 'NUTRACEUTICALS_Enhanced_with_Brands_and_Clusters.xlsx'
OUTPUT_FILE = 'NUTRACEUTICALS_Rearranged_Clusters.xlsx'
CACHE_FOLDER = './model_cache'

# Clustering parameters
DISTANCE_THRESHOLD = 0.5  # Higher for fewer, larger clusters
FUZZY_THRESHOLD = 70      # Lower for more grouping in fuzzy fallback
BATCH_SIZE = 32
SEARCH_COL = 'search'
SORT_COL = 'count'

# SSL bypass option (use cautiously)
BYPASS_SSL = False
if BYPASS_SSL:
    logger.warning("SSL verification disabled. Use only for testing.")
    ssl._create_default_https_context = ssl._create_unverified_context
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Load the multilingual sentence transformer model
model = None
try:
    logger.info("Loading multilingual sentence transformer model...")
    model = SentenceTransformer(
        'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
        cache_folder=CACHE_FOLDER,
        use_auth_token=False,
        verify_ssl=not BYPASS_SSL
    )
    logger.info("Model loaded successfully.")
except Exception as e:
    logger.error(f"Failed to load SentenceTransformer: {str(e)}")
    if FUZZY_AVAILABLE:
        logger.info("Falling back to rapidfuzz matching.")
    else:
        logger.warning("rapidfuzz not installed. No clustering if embeddings fail.")

# Process sheet with embeddings
def process_sheet_embeddings(df, search_col='search', sort_col='count', distance_threshold=0.5):
    if search_col not in df.columns:
        logger.warning(f"Column '{search_col}' not found. Skipping rearrangement.")
        return df
    
    searches = df[search_col].fillna('').astype(str).tolist()
    num_searches = len(searches)
    logger.info(f"Processing {num_searches} search terms (original text preserved)...")
    
    embeddings = model.encode(searches, show_progress_bar=True, batch_size=BATCH_SIZE)
    similarity_matrix = cosine_similarity(embeddings)
    distance_matrix = 1 - similarity_matrix
    
    clustering = AgglomerativeClustering(
        n_clusters=None,
        metric='precomputed',
        linkage='average',
        distance_threshold=distance_threshold
    )
    labels = clustering.fit_predict(distance_matrix)
    
    empty_mask = np.array(searches) == ''
    labels[empty_mask] = -1
    
    df['cluster_id'] = labels
    
    if sort_col in df.columns:
        df[sort_col] = pd.to_numeric(df[sort_col], errors='coerce').fillna(0)
    else:
        logger.warning(f"Column '{sort_col}' not found. Sorting by cluster_id only.")
        sort_col = None
    
    sort_cols = ['cluster_id']
    if sort_col:
        sort_cols.append(sort_col)
    df_sorted = df.sort_values(by=sort_cols, ascending=[True, False] if sort_col else [True])
    
    logger.info(f"Embedding-based clustering complete: {len(np.unique(labels))} clusters (threshold: {distance_threshold}).")
    return df_sorted

# Fallback: Fuzzy matching
def process_sheet_fuzzy(df, search_col='search', sort_col='count', threshold=70):
    if search_col not in df.columns:
        logger.warning(f"Column '{search_col}' not found. Skipping rearrangement.")
        return df
    
    searches = df[search_col].fillna('').astype(str).tolist()
    num_searches = len(searches)
    logger.info(f"Processing {num_searches} search terms with fuzzy matching...")
    
    dist_matrix = np.zeros((num_searches, num_searches))
    for i in range(num_searches):
        for j in range(i + 1, num_searches):
            sim = fuzz.partial_ratio(searches[i], searches[j])
            dist = 100 - sim
            dist_matrix[i, j] = dist / 100.0
            dist_matrix[j, i] = dist / 100.0
    
    clustering = AgglomerativeClustering(
        n_clusters=None,
        metric='precomputed',
        linkage='average',
        distance_threshold=(100 - threshold) / 100.0
    )
    labels = clustering.fit_predict(dist_matrix)
    
    empty_mask = np.array(searches) == ''
    labels[empty_mask] = -1
    
    df['cluster_id'] = labels
    
    if sort_col in df.columns:
        df[sort_col] = pd.to_numeric(df[sort_col], errors='coerce').fillna(0)
    else:
        logger.warning(f"Column '{sort_col}' not found. Sorting by cluster_id only.")
        sort_col = None
    
    sort_cols = ['cluster_id']
    if sort_col:
        sort_cols.append(sort_col)
    df_sorted = df.sort_values(by=sort_cols, ascending=[True, False] if sort_col else [True])
    
    logger.info(f"Fuzzy matching clustering complete: {len(np.unique(labels))} clusters (threshold: {threshold}).")
    return df_sorted

# Main processing function
def process_sheet(df, search_col='search', sort_col='count', distance_threshold=0.5):
    if model:
        try:
            return process_sheet_embeddings(df, search_col, sort_col, distance_threshold)
        except Exception as e:
            logger.warning(f"Embedding-based clustering failed: {str(e)}.")
            if FUZZY_AVAILABLE:
                logger.info("Falling back to rapidfuzz matching.")
                return process_sheet_fuzzy(df, search_col, sort_col, threshold=FUZZY_THRESHOLD)
            else:
                logger.warning("rapidfuzz not installed. Returning original DataFrame.")
                return df
    else:
        if FUZZY_AVAILABLE:
            logger.info("No embedding model. Using rapidfuzz matching.")
            return process_sheet_fuzzy(df, search_col, sort_col, threshold=FUZZY_THRESHOLD)
        else:
            logger.warning("No clustering possible. Returning original DataFrame.")
            return df

# Main function
def main():
    try:
        xls = pd.ExcelFile(INPUT_FILE)
        sheet_names = xls.sheet_names
        logger.info(f"Processing all sheets: {sheet_names}")
        
        with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
            for sheet_name in sheet_names:
                logger.info(f"Reading sheet: {sheet_name}")
                df = pd.read_excel(INPUT_FILE, sheet_name=sheet_name)
                
                df_rearranged = process_sheet(df, search_col=SEARCH_COL, sort_col=SORT_COL, distance_threshold=DISTANCE_THRESHOLD)
                df_rearranged.to_excel(writer, sheet_name=sheet_name, index=False)
                
                logger.info(f"Processed sheet: {sheet_name} (rows: {len(df_rearranged)}, clusters: {df_rearranged['cluster_id'].nunique()})")
        
        logger.info(f"\nOutput saved to: {OUTPUT_FILE}")
        logger.info("Rearrangement complete. Sorted by cluster_id (asc) and count (desc).")
        logger.info("Example: 'اومیجا ٣', 'اومیغا ٣', 'omega3' in same cluster.")
    
    except FileNotFoundError:
        logger.error(f"Input file '{INPUT_FILE}' not found.")
    except Exception as e:
        logger.error(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2025-09-30 16:05:24,506 - INFO - rapidfuzz available for enhanced fallback matching.
2025-09-30 16:05:24,511 - INFO - Loading multilingual sentence transformer model...
2025-09-30 16:05:24,513 - ERROR - Failed to load SentenceTransformer: SentenceTransformer.__init__() got an unexpected keyword argument 'verify_ssl'
2025-09-30 16:05:24,515 - INFO - Falling back to rapidfuzz matching.
2025-09-30 16:05:24,781 - INFO - Processing all sheets: ['queries_clustered', 'generic_type']
2025-09-30 16:05:24,813 - INFO - Reading sheet: queries_clustered
2025-09-30 16:05:30,202 - INFO - No embedding model. Using rapidfuzz matching.
2025-09-30 16:05:30,224 - INFO - Processing 15243 search terms with fuzzy matching...
2025-09-30 16:14:15,314 - INFO - Fuzzy matching clustering complete: 690 clusters (threshold: 70).
2025-09-30 16:14:24,622 - INFO - Processed sheet: queries_clustered (rows: 15243, clusters: 690)
2025-09-30 16:14:24,624 - INFO - Reading sheet: generic_type
2025-09-30 16:14:28,759 - INFO 

## STEP 3: Add Category Split + 3-Month Aggregated CTR / CR (In-Place)

In [10]:
"""
STEP 3: Add Category Split + 3-Month Aggregated CTR / CR (In-Place)
------------------------------------------------------------------
Updates ONLY the sheet 'queries_clustered' in:
    NUTRACEUTICALS_Enhanced_with_Brands_and_Clusters.xlsx

Actions:
  - Split 'category' into Department / Category / Sub Category / Class / Sub Class (delimiter: ///)
  - Aggregate 3-month totals per GROUP_BY_COLUMNS
  - Add columns:
      total_impressions_3m
      total_clicks_3m  
      total_conversions_3m
      ctr_3m
      cr_3m
  - Remove any pre-existing aggregate columns before recalculation
  - DO NOT touch other sheets
  - Creates a single backup once
"""
import os
import shutil
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# ---------------- CONFIG ----------------
FILE_PATH = "NUTRACEUTICALS_Rearranged_Clusters.xlsx"
TARGET_SHEET = "queries_clustered"  # Change if your sheet has different name
BACKUP_BEFORE_OVERWRITE = True
GROUP_BY_COLUMNS = ["search","Department","Category","Sub Category"]  # Updated as requested

REQUIRED_CORE = [
    "category","search","count","Clicks","Conversions",
    "start_date","end_date"
]

NUMERIC_COLS = ["count","Clicks","Conversions"]

AGG_COLS = [
    "total_impressions_3m",
    "total_clicks_3m",
    "total_conversions_3m",
    "ctr_3m",
    "cr_3m"
]

def ensure_core(df: pd.DataFrame):
    """Check if all required columns exist"""
    missing = [c for c in REQUIRED_CORE if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required core columns: {missing}")

def split_category_path(val):
    """
    Split category path with /// delimiter
    Returns: (Department, Category, Sub Category, Class, Sub Class)
    """
    if not isinstance(val, str):
        return (None, None, None, None, None)
    
    val = val.strip()
    parts = [p.strip() for p in val.split("///")]
    parts = [p for p in parts if p]  # Remove empty parts
    
    # Pad with None to ensure we have 5 elements
    while len(parts) < 5:
        parts.append(None)
    
    return tuple(parts[:5])  # Return first 5 elements

def add_category_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Add Department, Category, Sub Category, Class, Sub Class columns"""
    # Check if columns already exist
    new_cols = ["Department","Category","Sub Category","Class","Sub Class"]
    if all(col in df.columns for col in new_cols):
        print("[INFO] Category columns already exist, skipping split")
        return df
    
    print("[INFO] Splitting category column into components...")
    
    departments, categories, sub_categories, classes, sub_classes = [], [], [], [], []
    
    for val in df["category"]:
        dept, cat, sub_cat, cls, sub_cls = split_category_path(val)
        departments.append(dept)
        categories.append(cat)
        sub_categories.append(sub_cat)
        classes.append(cls)
        sub_classes.append(sub_cls)
    
    df["Department"] = departments
    df["Category"] = categories
    df["Sub Category"] = sub_categories
    df["Class"] = classes
    df["Sub Class"] = sub_classes
    
    return df

def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Convert numeric columns to proper numeric types"""
    for col in NUMERIC_COLS:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
    return df

def drop_old_agg_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Remove existing aggregate columns before recalculation"""
    existing = [c for c in AGG_COLS if c in df.columns]
    if existing:
        print(f"[INFO] Removing existing aggregate columns: {existing}")
        df = df.drop(columns=existing)
    return df

def aggregate_three_months(df: pd.DataFrame) -> pd.DataFrame:
    """Calculate 3-month aggregates grouped by specified columns"""
    print(f"[INFO] Aggregating by: {GROUP_BY_COLUMNS}")
    
    agg = (df.groupby(GROUP_BY_COLUMNS, dropna=False)
             .agg(
                 total_impressions_3m=("count","sum"),
                 total_clicks_3m=("Clicks","sum"),
                 total_conversions_3m=("Conversions","sum")
             ).reset_index())
    
    # Calculate rates
    agg["ctr_3m"] = np.where(
        agg["total_impressions_3m"] > 0,
        agg["total_clicks_3m"] / agg["total_impressions_3m"],
        0
    )
    
    agg["cr_3m"] = np.where(
        agg["total_impressions_3m"] > 0,
        agg["total_conversions_3m"] / agg["total_impressions_3m"],
        0
    )
    
    print(f"[INFO] Created {len(agg)} aggregate groups")
    return agg

def merge_aggregates(df: pd.DataFrame, agg: pd.DataFrame) -> pd.DataFrame:
    """Merge aggregate data back to main dataframe"""
    return df.merge(agg, on=GROUP_BY_COLUMNS, how="left")

def reorder_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Reorder columns for better readability"""
    preferred = [
        "category","Department","Category","Sub Category","Class","Sub Class",
        "Brand","search","count","Clicks","Conversions",
        "clickThroughRate","conversionRate",
        "total_impressions_3m","total_clicks_3m","total_conversions_3m",
        "ctr_3m","cr_3m",
        "averageClickPosition","underperforming",
        "start_date","end_date","Cluster ID","cluster_id"
    ]
    
    existing_pref = [c for c in preferred if c in df.columns]
    others = [c for c in df.columns if c not in existing_pref]
    
    return df[existing_pref + others]

def write_sheet_in_place(path: str, sheet_name: str, df: pd.DataFrame):
    """Write dataframe to specific sheet in existing workbook"""
    wb = load_workbook(path)
    
    if sheet_name not in wb.sheetnames:
        raise ValueError(f"Sheet '{sheet_name}' not found in workbook. Available sheets: {wb.sheetnames}")
    
    # Remove old sheet and create new one
    ws = wb[sheet_name]
    wb.remove(ws)
    ws_new = wb.create_sheet(title=sheet_name)
    
    # Write data
    for r in dataframe_to_rows(df, index=False, header=True):
        ws_new.append(r)
    
    wb.save(path)

def process_target_sheet():
    """Main processing function"""
    if not os.path.exists(FILE_PATH):
        raise FileNotFoundError(f"File not found: {FILE_PATH}")

    # Create backup once
    if BACKUP_BEFORE_OVERWRITE:
        backup_path = FILE_PATH.replace(".xlsx", "_backup_step3.xlsx")
        if not os.path.exists(backup_path):
            shutil.copy2(FILE_PATH, backup_path)
            print(f"[INFO] Backup created: {backup_path}")
        else:
            print(f"[INFO] Backup already exists: {backup_path}")

    # Load target sheet
    try:
        df = pd.read_excel(FILE_PATH, sheet_name=TARGET_SHEET)
        print(f"[INFO] Loaded {len(df)} rows from sheet '{TARGET_SHEET}'")
    except ValueError as e:
        raise RuntimeError(f"Cannot load sheet '{TARGET_SHEET}': {e}")

    # Process data
    ensure_core(df)
    df = coerce_numeric(df)
    df = add_category_columns(df)
    df = drop_old_agg_columns(df)
    
    # Remove any unwanted columns
    unwanted_cols = ["Index_language"]
    for col in unwanted_cols:
        if col in df.columns:
            df = df.drop(columns=[col])
            print(f"[INFO] Removed column: {col}")
    
    # Calculate aggregates
    agg = aggregate_three_months(df)
    df = merge_aggregates(df, agg)
    df = reorder_columns(df)
    
    # Write back to file
    write_sheet_in_place(FILE_PATH, TARGET_SHEET, df)
    
    print(f"[DONE] Sheet '{TARGET_SHEET}' updated in workbook: {FILE_PATH}")
    print(f"[INFO] Final dataset has {len(df)} rows and {len(df.columns)} columns")
    
    # Show sample of new columns
    if len(df) > 0:
        print("\n[INFO] Sample of new aggregate columns:")
        sample_cols = ["search","total_impressions_3m","total_clicks_3m","ctr_3m","cr_3m"]
        available_cols = [c for c in sample_cols if c in df.columns]
        print(df[available_cols].head(3).to_string(index=False))

# Execute
if __name__ == "__main__":
    try:
        process_target_sheet()
    except Exception as e:
        print(f"[ERROR] {e}")
        raise


[INFO] Backup created: NUTRACEUTICALS_Rearranged_Clusters_backup_step3.xlsx
[INFO] Loaded 15243 rows from sheet 'queries_clustered'
[INFO] Splitting category column into components...
[INFO] Aggregating by: ['search', 'Department', 'Category', 'Sub Category']
[INFO] Created 5253 aggregate groups
[DONE] Sheet 'queries_clustered' updated in workbook: NUTRACEUTICALS_Rearranged_Clusters.xlsx
[INFO] Final dataset has 15243 rows and 24 columns

[INFO] Sample of new aggregate columns:
 search  total_impressions_3m  total_clicks_3m  ctr_3m    cr_3m
hemadid                  1795            352.0  0.1961 0.103621
hemadid                  1795            352.0  0.1961 0.103621
hemadid                  1795            352.0  0.1961 0.103621
